In [1]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import time,datetime

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import logging
import torch
logger = logging.getLogger(__name__)

In [3]:
def flatten_features(x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features
class SimpNet(nn.Module):
    def __init__(self, size_data, n_classes):
        super(SimpNet, self).__init__()

        self.size_data = np.copy(size_data)
        self.conv1 = nn.Conv2d(1, 30, 3, stride=(1, 1), padding=(1, 1))
        self.pool1 = nn.MaxPool2d((2, 2), stride=(2, 2))


        self.size_data //= 2

        #####################
        ####### Second ######
        #####################
        self.conv2 = nn.Conv2d(30, 60, 3, stride=(1, 1), padding=(1, 1))
        self.pool2 = nn.MaxPool2d((2, 2), stride=(2, 2))


        self.size_data //= 2

        ####################
        ####### Third ######
        ####################
        self.conv3 = nn.Conv2d(60, 80, 3, stride=(1, 1), padding=(1, 1))
        self.pool3 = nn.MaxPool2d((2, 2), stride=(2, 2))

        self.size_data //= 2

        #####################
        ####### FC ##########
        #####################
        self.fc1 = nn.Linear(80*self.size_data[0]*self.size_data[1], 500)
        
        self.fc2 = nn.Linear(500, n_classes)
        self.final = nn.Softmax(1)

    def forward(self, image):

        ####################
        ####### First ######
        ####################
        image = self.pool1(F.relu(self.conv1(image)))


        #####################
        ####### Second ######
        #####################
        image = self.pool2(F.relu(self.conv2(image)))

        ####################
        ####### Third ######
        ####################
        image = self.pool3(F.relu(self.conv3(image)))
      

        #####################
        ####### FC ##########
        #####################
        image = image.view(-1, flatten_features(image))

        image = F.relu(self.fc1(image))
        image = F.relu(self.fc2(image))

        label = self.final(image)

        return label

In [4]:
transform = transforms.Compose([transforms.Resize((256,256)),
                               transforms.Grayscale(num_output_channels=1),
                               transforms.ToTensor(),
                               ])

train_set = ImageFolder('./covidx_image_folder/train/', transform= transform)
valid_set = ImageFolder('./covidx_image_folder/validation/', transform= transform)

train_loader_folder = DataLoader(train_set, batch_size=24, shuffle=True)
valid_loader_folder = DataLoader(valid_set, batch_size=24, shuffle=False)

In [5]:
def make_train_figure(loss_train, loss_val, acc_train, acc_val, path):
    plt.plot(loss_train, label='Loss Train')
    plt.plot(loss_val, label='Loss Val')
    plt.plot(acc_train, label='Acc Train')
    plt.plot(acc_val, label='Acc Val')
    plt.legend()
    plt.savefig(path)
    plt.show()

In [6]:
def save_model(model, optimizer, epoch, dict_of_values):
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'dict_of_values': dict_of_values}, os.path.join('.', 'model.tar'))

In [7]:
def train_epoch(epoch, model, data_loader, optimizer, criterion):
    model.train()
    N = len(data_loader.dataset)
    start_time = time.time()
    aLoss = 0
    Acc = 0

    for batch_idx, batch in enumerate(tqdm(data_loader)):
        image,label = batch
        image = image.cuda()
        label = label.cuda()

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        aLoss += loss.item()
        Acc += output.data.max(1)[1].eq(label.data).cpu().sum().numpy()
       

    aLoss /= (batch_idx + 1)
    Acc /= N
    print('Training: [Epoch %4d] loss: %.4f accuracy: %.4f lr: %.6f' % (epoch, aLoss, Acc, 0.001))
    logger.info('Training: [Epoch %4d] loss: %.4f accuracy: %.4f lr: %.6f' % (epoch, aLoss, Acc, 0.001))

    return aLoss, Acc

In [8]:
def validation_epoch(epoch, model, data_loader, criterion):
    model.eval()
    N = len(data_loader.dataset)
    aLoss = 0
    Acc = 0

    for batch_idx, batch in enumerate(tqdm(data_loader)):
       
        image,label = batch

        image = image.cuda()
        label = label.cuda()

        output = model(image)
        pred = output.cpu().data.numpy()
        pred_argmax = np.argmax(pred, axis=1)

        aLoss += criterion(output, label).item()
        Acc += output.data.max(1)[1].eq(label.data).cpu().sum().numpy()

    aLoss /= (batch_idx + 1)
    Acc /= N
    print('Evaluation: [Epoch %4d] loss: %.4f accuracy: %.4f' % (epoch, aLoss, Acc))
    logger.info('Evaluation: [Epoch %4d] loss: %.4f accuracy: %.4f' % (epoch, aLoss, Acc))

    return pred_argmax, aLoss, Acc


In [9]:
def train_model(model, train_loader, validation_loader):
    start_time = time.time()

    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.SGD(
        model.parameters(), 
        lr=0.001, 
        momentum=0.5, 
        weight_decay=0.005, 
        nesterov=True
    )

    # For plot
    loss_train = []
    loss_val = []
    acc_val = []
    acc_train = []
    max_acc = -1
    acc_val_ = 1
    min_loss_train = 1000
    min_loss_val = 1000
    epoch_start = 1

    for epoch in range(epoch_start, 5+1):

        # Train and validation step and save loss and acc for plot
        loss_train_, acc_train_ = train_epoch(epoch, model, train_loader, optimizer, criterion)
        _, loss_val_, acc_val_ = validation_epoch(epoch, model, validation_loader, criterion)

        loss_train.append(loss_train_)
        acc_train.append(acc_train_)
        loss_val.append(loss_val_)
        acc_val.append(acc_val_)

        #wait_change_lr += 1

        # Best model saved
        # if (acc_val_ > max_acc) or (acc_val_ >= max_acc and loss_train_ < min_loss_train):
        if min_loss_val > loss_val_:
            save_model(model, optimizer=optimizer, epoch=epoch, dict_of_values={'loss_train_': loss_train_, 'acc_train_': acc_train_, 'loss_val_': loss_val_, 'acc_val_': acc_val_})
            max_acc = acc_val_
            min_loss_val = loss_val_
            min_loss_train = loss_train_


    logger.info('Trained with %d epochs, lr = %g, batchsize = %d, momentum = %g with max validation accuracy of %.2f done in %s' %\
        (100, 0.001, 16,0.5, max_acc, datetime.timedelta(seconds=int(time.time() - start_time))))

    make_train_figure(loss_train, loss_val, acc_train, acc_val, os.path.join('.', 'Train.png'))

In [10]:
model = SimpNet(np.array([256,256]),3)
model.cuda()
train_model(model,train_loader_folder,valid_loader_folder)

  0%|          | 0/59 [00:00<?, ?it/s]

Training: [Epoch    1] loss: 1.0986 accuracy: 0.0367 lr: 0.001000


100%|██████████| 59/59 [00:29<00:00,  2.00it/s]


Evaluation: [Epoch    1] loss: 1.0986 accuracy: 0.0401


  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/59 [00:00<?, ?it/s]

Training: [Epoch    5] loss: 1.0986 accuracy: 0.0367 lr: 0.001000


100%|██████████| 59/59 [00:29<00:00,  2.01it/s]

Evaluation: [Epoch    5] loss: 1.0986 accuracy: 0.0401


NameError: name 'datetime' is not defined